In [ ]:
knitr::opts_chunk$set(echo = TRUE)



## Target
The goal of this competition is to predict rainfall for each day of the year.

ML tools: 
  tidymodels related worksflows & glm, lightgbm, ranger engine.
Evaluatioin metric:
  roc_auc, percent of rain probability.
  sample_submission.csv : 0.5. it is value is 0. I tried change it to 1, the score is 0.5 as well.
  kaggle best score is 0.87 in the begining.

## librar y & load_data

### library


In [ ]:
library(tidyverse)
library(tidymodels)
library(future)
library(purrr)
library(furrr)
library(textrecipes)
library(themis)

library(bonsai)
library(lightgbm)
library(xgboost)
library(ranger)

library(readr)
library(janitor)
library(lubridate)


In [ ]:
#TODO, column 18,7 has some strange data leave it handle future. 71 lines in train, 159 line in test
# row833049 column 18 is f has-ring is total wrong. 
# so keep using readr:read_csv handling


### loading data



In [ ]:
data_path <- '../input/playground-series-s5e3/'
train<- 
  readr::read_csv(file.path(data_path, 'train.csv'),
                  show_col_types = FALSE)|>
  mutate(rainfall=as.factor(rainfall))
test <- 
  readr::read_csv(file.path(data_path, 'test.csv'),
                  show_col_types = FALSE)
submission <-  readr::read_csv(file.path(data_path, 'sample_submission.csv'),show_col_types = FALSE)


### quick skim



In [ ]:
train|> skimr::skim()



In [ ]:
test|> skimr::skim()



In [ ]:
submission |> skimr::skim()



### check if train & test is same distribution



In [ ]:
get_df_var<-function(df){
  df|>
    select(-any_of(c('id','rainfall')))|>
    summarize_all(var)|>
    pivot_longer(cols=everything(),
                 names_to='feature',
                 values_to='variance')

}
list(train, test)|>
  map_dfr(\(x) get_df_var(x), .id = "dataset") |>
  pivot_wider(names_from=dataset, values_from = variance)|>
  mutate(diff=`1`-`2`)|>arrange(desc(abs(diff)))


### Finding of different distribution
- the day feature (-49)
- cloud  (3.32)
- pressure, mintemp, windspeed, dewpoint


## coding

### 1. Data Loading and Initial Exploration ----

### 2. Feature Engineering ----

-   leave it in the preprocessing recipe

### 3. Data Splitting ----

#### augment_df

#### split/cv


In [ ]:
set.seed(1234)

df_split <- initial_split(train, prop = 0.8, strata = rainfall)
 train_set <- training(df_split)
 test_set <- testing(df_split)
cv_folds <- vfold_cv(train_set,v = 10,strata=rainfall)


### 4. Preprocessing Recipe ----

#### 4.1 v0 base_line - 


In [ ]:
rcp_bs_v0 <-
  recipe(rainfall ~ ., data = train_set) |>
  update_role(id, new_role='ID')|>
  step_impute_median(all_numeric_predictors())|> 
  step_log(all_numeric_predictors(),offset = 1, skip = FALSE) |>
  # process the logical feature to factor feature
  step_bin2factor(all_logical_predictors())|>
  # proceeding the convert the character feature to factorical feature
  step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors()) |>
  step_other(all_nominal_predictors())|>
  step_dummy(all_nominal_predictors(),one_hot = TRUE) |>
  step_nzv(all_predictors())|>
  #step_corr(all_numeric_predictors())|>
  step_normalize(all_numeric_predictors())|> # Scale numeric predictors
  #step_smote(rainfall,over_ratio = 1,skip = TRUE)|>
  check_missing(all_predictors())


#### 4.10 all recipes



In [ ]:
set.seed(1234)
library(future)
library(furrr)
selected_rcps <- list(base=rcp_bs_v0)
plan(multisession,workers = 5)
selected_rcps|>map(\(rcp_item) rcp_item|>prep()|>bake(new_data=train)|>summary())
plan(sequential)


### 5. Model Specification ----



In [ ]:
glm_eng <- 
  logistic_reg(penalty = 0.01,
               mixture = 0.5) |>  # Example penalty and mixture values
  set_engine("glmnet") |>
  set_mode("classification")    # Specify classification

lgbm_eng<-
   parsnip::boost_tree(
      trees = 500, # Number of trees
      learn_rate = 0.01,
      tree_depth = 6,
      loss_reduction = 0.001,
      sample_size = 0.85, # Added sample_size
      #tree_depth = tune(),
      
      #mtry = tune(),
      min_n = 30
   ) |>
   set_mode("classification")|>
   set_engine("lightgbm",
              #metric='roc_auc', 
              num_leaves = 20,
              num_threads = 4,
              verbose=1) 

rf_eng<- rand_forest( trees = 500, 
                      #learn_rate = 0.01,
                      #mtry=tune(), 
                      min_n=30) |>
  set_engine("ranger",num.threads=4)|>
  set_mode("classification") 

xgb_eng<- parsnip::boost_tree( trees = 500, 
                      learn_rate = 0.01,
                      loss_reduction = 0.001,
                      sample_size = 0.85, # Added sample_size
                      #mtry=tune(),
                      min_n=30) |>
  set_engine("xgboost",num.threads=8)|>
  set_mode("classification")
selected_eng <- list(glm=glm_eng,
                     rf=rf_eng,
                     lgbm=lgbm_eng#,
                     #xgb=xgb_en
                     )


### 6. Workflow ----
#### set metrics


In [ ]:
rocauc_metrics <- metric_set(accuracy, roc_auc) # main goal is roc_auc, accuracy is just for reference



#### simple wflow



In [ ]:
set.seed(1234)
simple_wf_fit <-
  workflow() |>
  add_recipe(rcp_bs_v0) |>
  add_model(lgbm_eng)|>
  last_fit(df_split,
           metrics=rocauc_metrics)

simple_wf_fit|>collect_metrics()
# simple_wf_fit |>
#   extract_fit_engine()|>
#   plot()


#### simple workflowset



In [ ]:
set.seed(1234)
library(future)
plan(multisession,workers = 12)
ctrl <- control_resamples(save_pred = TRUE, save_workflow = TRUE,verbose=TRUE)
wfs_result <-
  workflow_set(preproc = selected_rcps,
               models = selected_eng ) |>
  workflow_map(fn='fit_resamples',
               resamples =cv_folds,
               metrics =rocauc_metrics,
               control = ctrl
               )
wfs_result|>collect_metrics()  |> filter(.metric=='roc_auc')
  
plan(sequential)


### 7 stacking



In [ ]:
set.seed(1234)
library(future)
#plan(multisession,workers = 4)
combined_fit <-
  stacks::stacks()|>
  stacks::add_candidates(wfs_result)|>
  stacks::blend_predictions()|>
  stacks::fit_members()

combined_fit|>
  autoplot(type = "weights")

autoplot(combined_fit)
#plan(sequential)


### 7. Tuning Grid ----



In [ ]:
# cars_grid <- grid_space_filling(
#   #learn_rate(range = c(0.01, 0.1)),
#   # loss_reduction(range = c(0, 10)), #Keep if you want it.
#   #bag_fraction(range = c(0.7, 0.9)), # Specify prop = TRUE.
#   tree_depth(range = c(5, 10)),
#   finalize(mtry(range = c(2, 10)),
#            select(cars_train_set, -Price)),
#   min_n(range = c(2, 20)),
#   size = 10
# )


### 8. Cross-Validation ----



In [ ]:
# combined it with step3 data splitting



### 9. Tuning and Evaluation ----



In [ ]:
# plan(multisession,workers =2)
# cars_tune_results <- cars_workflow |>
#   tune_grid(
#     resamples = cars_folds,
#     grid = cars_grid,
#     metrics = metric_set(rmse),
#      control = control_grid(save_pred = TRUE, 
#                             verbose = TRUE,
#                             allow_par = F) # Keep predictions
#   )
#  
#  # Find best parameters
#  best_params <- cars_tune_results |>
#    select_best("rmse")
# 
#  # Finalize workflow with best parameters
#  final_workflow <- cars_workflow |>
#    finalize_workflow(best_params)


In [ ]:
# Fit the final workflow to the training data
# final_lgbm_fit <- last_fit(final_workflow,cars_split )
# final_lgbm_mod <- extract_workflow(final_lgbm_fit )
# collect_metrics(final_lgmb_mod)

# plan(sequential)


### 10. Evaluate on Test Set ----



In [ ]:
combined_test_result <- 
  test_set %>%
  bind_cols(predict(combined_fit, new_data=test_set,type='prob'))
combined_test_result|>roc_auc(rainfall, .pred_1,event_level = 'second')


### 11. Prepare Submission ----



In [ ]:
set.seed(1234)
library(future)
plan(multisession,workers = 12)
final_model <- combined_fit#simple_wf_fit|>extract_workflow()
final_predictions <- final_model |>
   predict(new_data = test,type='prob') 
plan(sequential)

 # #Handle negative predictions
 # final_predictions <- final_predictions |>
 #   mutate(.pred= ifelse(.pred< 0, 0, .pred))

 # Save submission file
 submission |>
   mutate(rainfall=final_predictions$.pred_1)|>
   readr::write_csv("submission.csv")
 zip('submission.csv.zip','submission.csv')
 


## kaggle submission



### score submit


In [ ]:
# submit latest submission.csv
system('kaggle competitions submit -c playground-series-s5e3 -f submission.csv.zip -m "sample_submission"')
Sys.sleep(15)
# get latest score 
system('kaggle competitions submissions -q -c playground-series-s5e3')
# 
# # get leader board score
# #system('kaggle competitions leaderboard -s -v -c playground-series-s4e8')



### notebook convert


In [ ]:
library(rmd2jupyter)
rmd2jupyter('rainfall_roc_auc_workflow.Rmd')
